# **Profit & Sensitvity Analysis**

## MSDS Group 2 Final Project
### Hassan Ali, Ethan Norton, Jamia Russell

In [1]:
import os

def find_glpsol():
    # Common directories to search
    possible_directories = [
        os.path.expanduser("~/Anaconda3/Library/bin/"),  # Conda default location
        os.path.expanduser("~/Anaconda3/envs/"),         # Conda environments
        "C:\glpk-4.65\w64",                    # Typical Program Files location
        "C:/Program Files (x86)/GLPK/bin/",              # Another potential Program Files location
    ]
    
    # Search through Conda environments
    conda_env_path = os.path.expanduser("~/Anaconda3/envs/")
    if os.path.exists(conda_env_path):
        for env in os.listdir(conda_env_path):
            possible_directories.append(os.path.join(conda_env_path, env, "Library/bin/"))
    
    # Check if glpsol.exe exists in any of the possible directories
    for directory in possible_directories:
        glpsol_path = os.path.join(directory, "glpsol.exe")
        if os.path.isfile(glpsol_path):
            return glpsol_path

    # Search system PATH as a last resort
    for path in os.environ["PATH"].split(os.pathsep):
        glpsol_path = os.path.join(path, "glpsol.exe")
        if os.path.isfile(glpsol_path):
            return glpsol_path

    return None

# Run the script
glpsol_path = find_glpsol()

if glpsol_path:
    print(f"Found glpsol.exe at: {glpsol_path}")
else:
    print("glpsol.exe not found on the system.")


Found glpsol.exe at: C:\glpk-4.65\w64\glpsol.exe


In [2]:
#Original Model w/ Sensitivity Analysis Report
import pulp
from pulp import LpProblem, LpMaximize, LpVariable, GLPK_CMD, LpStatus

# Define raw materials and products
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Define data as arrays
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Define raw material usage per product (rows: raw materials, columns: products)
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.00],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

# Define labor hours per product at each site
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

# Define the problem
prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

# Define decision variables for the quantity of each product to produce at each site
product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]

# Calculate profit per product per site
profit_values_site1 = []
profit_values_site2 = []

for i in range(len(products)):
    cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
    profit_values_site1.append(product_prices[i] - cost)
    profit_values_site2.append(product_prices[i] - cost)

profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]

# Objective function
prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

# Calculate total labor hours per product per site
labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

# Sum the total labor hours for each site
sum_labor_site1 = pulp.lpSum(labor_hours_site1)
sum_labor_site2 = pulp.lpSum(labor_hours_site2)

# Add labor hours constraints
prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

# Calculate inventory constraints per raw material per site
for i in range(len(raw_materials)):
    # Constraint for Site 1
    prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
    
    # Constraint for Site 2
    prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

# Solve the problem using GLPK
glpk_path = "C:\glpk-4.65\w64\glpsol.exe" 
status = prob.solve(GLPK_CMD(path=glpk_path))

# Output results
optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
objective_value = pulp.value(prob.objective)

# Calculate inventory used per raw material
inventory_used_site1 = {raw_materials[i]: sum(optimal_values_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}
inventory_used_site2 = {raw_materials[i]: sum(optimal_values_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}

# Calculate total labor used
total_labor_used_site1 = sum([optimal_values_site1[i] * site1_labor_hours[i] for i in range(len(products))])
total_labor_used_site2 = sum([optimal_values_site2[i] * site2_labor_hours[i] for i in range(len(products))])
import pulp
from pulp import GLPK_CMD, LpProblem, LpMaximize, LpVariable, LpStatus

# Define raw materials and products
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Define data as arrays
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Define raw material usage per product (rows: raw materials, columns: products)
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.00],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

# Define labor hours per product at each site
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

# Define the problem
prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

# Define decision variables for the quantity of each product to produce at each site
product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]

# Calculate profit per product per site
profit_values_site1 = []
profit_values_site2 = []

for i in range(len(products)):
    cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
    profit_values_site1.append(product_prices[i] - cost)
    profit_values_site2.append(product_prices[i] - cost)

profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]

# Objective function
prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

# Calculate total labor hours per product per site
labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

# Sum the total labor hours for each site
sum_labor_site1 = pulp.lpSum(labor_hours_site1)
sum_labor_site2 = pulp.lpSum(labor_hours_site2)

# Add labor hours constraints
prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

# Calculate inventory constraints per raw material per site
for i in range(len(raw_materials)):
    # Constraint for Site 1
    prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
    
    # Constraint for Site 2
    prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

# Solve the problem using GLPK
glpk_path = "C:\glpk-4.65\w64\glpsol.exe"
status = prob.solve(GLPK_CMD(path=glpk_path, options=['--ranges', 'Product_Optimization.sen']))

# Output results
optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
objective_value = pulp.value(prob.objective)

# Calculate inventory used per raw material
inventory_used_site1 = {raw_materials[i]: sum(optimal_values_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}
inventory_used_site2 = {raw_materials[i]: sum(optimal_values_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}

# Calculate total labor used
total_labor_used_site1 = sum([optimal_values_site1[i] * site1_labor_hours[i] for i in range(len(products))])
total_labor_used_site2 = sum([optimal_values_site2[i] * site2_labor_hours[i] for i in range(len(products))])

# Print results
print("Optimal production values for Site 1:", optimal_values_site1)
print("Optimal production values for Site 2:", optimal_values_site2)
print("Total Profit:", objective_value)

print("Inventory used per raw material at Site 1:")
for material, usage in inventory_used_site1.items():
    print(f"{material}: {usage}")

print("Inventory used per raw material at Site 2:")
for material, usage in inventory_used_site2.items():
    print(f"{material}: {usage}")

print("Total labor used at Site 1:", total_labor_used_site1)
print("Total labor used at Site 2:", total_labor_used_site2)

# To extract sensitivity analysis, read the generated .sen file
try:
    with open("Product_Optimization.sen", "r") as f:
        sensitivity_analysis = f.read()
        print("\nSensitivity Analysis:\n")
        print(sensitivity_analysis)
except FileNotFoundError:
    print("Sensitivity analysis file not found. Ensure GLPK has generated the sensitivity report.")

# Print results
print("Optimal production values for Site 1:", optimal_values_site1)
print("Optimal production values for Site 2:", optimal_values_site2)
print("Total Profit:", objective_value)

print("Inventory used per raw material at Site 1:")
for material, usage in inventory_used_site1.items():
    print(f"{material}: {usage}")

print("Inventory used per raw material at Site 2:")
for material, usage in inventory_used_site2.items():
    print(f"{material}: {usage}")

print("Total labor used at Site 1:", total_labor_used_site1)
print("Total labor used at Site 2:", total_labor_used_site2)

Optimal production values for Site 1: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
Optimal production values for Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 119.529, 0.0, 152.381, 156.863]
Total Profit: 120974.44818499999
Inventory used per raw material at Site 1:
RSXX: 3000.0013500000005
STYY: 1839.37125
CBAA: 672.5223000000001
CBZZ: 214.7769
SBRXL: 0.0
SLSM: 0.0
CARBON 10: 1500.0016500000002
CARBON 20: 0.0
CARBON 30: 800.0002499999999
CARBON 40: 800.0013
CARBON 60: 0.0
Inventory used per raw material at Site 2:
RSXX: 3074.628
STYY: 571.42875
CBAA: 342.85725
CBZZ: 233.08155
SBRXL: 0.0
SLSM: 0.0
CARBON 10: 609.5979
CARBON 20: 0.0
CARBON 30: 800.0002499999999
CARBON 40: 800.0013
CARBON 60: 0.0
Total labor used at Site 1: 167.1882
Total labor used at Site 2: 167.9999

Sensitivity Analysis:

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Total_Profit = 120974.395 (MAXimum)

   No. Ro

In [3]:
#Profit Per Site & Profit Per Product Per Site

# Calculate total profit per site
total_profit_site1 = sum([optimal_values_site1[i] * profit_values_site1[i] for i in range(len(products))])
total_profit_site2 = sum([optimal_values_site2[i] * profit_values_site2[i] for i in range(len(products))])

# Calculate Profit Per Product Per Site 

# Site 1
site1pppresult = list([optimal_values_site1[i] * profit_values_site1[i] for i in range(len(products))])
print(site1pppresult)

# Site 2
site2pppresult = list([optimal_values_site2[i] * profit_values_site2[i] for i in range(len(products))])
print(site2pppresult)

# Calculate inventory used per raw material
inventory_used_site1 = {raw_materials[i]: sum(optimal_values_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}
inventory_used_site2 = {raw_materials[i]: sum(optimal_values_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}

# Calculate total labor used
total_labor_used_site1 = sum([optimal_values_site1[i] * site1_labor_hours[i] for i in range(len(products))])
total_labor_used_site2 = sum([optimal_values_site2[i] * site2_labor_hours[i] for i in range(len(products))])

# Output total profit per site
print(f"Total profit for Site 1: ${total_profit_site1:.2f}")
print(f"Total profit for Site 2: ${total_profit_site2:.2f}")

[0.0, 0.0, 7807.14462, 0.0, 0.0, 11160.138149999999, 0.0, 20918.101775, 24029.842969999998]
[0.0, 0.0, 0.0, 0.0, 0.0, 12111.275924999998, 0.0, 20918.101775, 24029.842969999998]
Total profit for Site 1: $63915.23
Total profit for Site 2: $57059.22


In [4]:
#Profit Per Material Per Site

profit_per_material_site1 = {}
profit_per_material_site2 = {}

for i in range(len(raw_materials)):
    revenue_site1 = sum(optimal_values_site1[j] * product_prices[j] * raw_material_usage[i][j] for j in range(len(products)))
    cost_site1 = inventory_used_site1[raw_materials[i]] * raw_material_costs[i]
    profit_per_material_site1[raw_materials[i]] = revenue_site1 - cost_site1
    
    revenue_site2 = sum(optimal_values_site2[j] * product_prices[j] * raw_material_usage[i][j] for j in range(len(products)))
    cost_site2 = inventory_used_site2[raw_materials[i]] * raw_material_costs[i]
    profit_per_material_site2[raw_materials[i]] = revenue_site2 - cost_site2


print("Profit per material for Site 1:", profit_per_material_site1)
print("Profit per material for Site 2:", profit_per_material_site2)

print("-" *100)
#ckecking work that profit per material adds correctly
total_profit_site1 = sum(profit_per_material_site1.values())
total_profit_site2 = sum(profit_per_material_site2.values())

print("Total profit for Site 1:", total_profit_site1)
print("Total profit for Site 2:", total_profit_site2)


Profit per material for Site 1: {'RSXX': 32340.633710000002, 'STYY': 14015.996000000001, 'CBAA': 5457.99372, 'CBZZ': 1911.5144099999998, 'SBRXL': 0.0, 'SLSM': 0.0, 'CARBON 10': -877.5890249999993, 'CARBON 20': 0.0, 'CARBON 30': 4533.334749999999, 'CARBON 40': 6533.343950000002, 'CARBON 60': 0.0}
Profit per material for Site 2: {'RSXX': 33057.04955, 'STYY': 6323.8115, 'CBAA': 3622.8582749999996, 'CBZZ': 2074.4257949999997, 'SBRXL': 0.0, 'SLSM': 0.0, 'CARBON 10': 914.3968500000001, 'CARBON 20': 0.0, 'CARBON 30': 4533.334749999999, 'CARBON 40': 6533.343950000002, 'CARBON 60': 0.0}
----------------------------------------------------------------------------------------------------
Total profit for Site 1: 63915.227515
Total profit for Site 2: 57059.22067000001


In [5]:
#Sensitivity Analysis on Labor Constraints a Site 1

#original
original_labor_hours_site1 = 168
original_labor_hours_site2 = 168

#constraint changes
for adjustment in range(-20, 21, 1):  #adjusting labor hours by -20, -10, 0, 10, 20
    new_labor_hours_site1 = original_labor_hours_site1 + adjustment
    prob.constraints["Site1_Labor_Constraint"].changeRHS(new_labor_hours_site1)
    
    #solve
    prob.solve(GLPK_CMD(path=glpk_path))
    
    #output
    new_optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
    new_objective_value = pulp.value(prob.objective)
    
    print(f"\nAdjustment of {adjustment} labor hours at Site 1:")
    print("New Optimal Production Plan for Site 1:", new_optimal_values_site1)
    print("New Total Profit:", new_objective_value)


Adjustment of -20 labor hours at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 114.236, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 118442.72204499999

Adjustment of -19 labor hours at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 117.093, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 118574.65830499999

Adjustment of -18 labor hours at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 119.95, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 118706.59456499999

Adjustment of -17 labor hours at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 122.807, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 118838.530825

Adjustment of -16 labor hours at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 125.664, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 118970.467085

Adjustment of -15 labor hours at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 128.521, 0.0,

In [6]:
#Sensitivity Analysis on Labor Constraints at Site 2

#original
original_labor_hours_site1 = 168
original_labor_hours_site2 = 168

#constraint changes
for adjustment in range(-20, 21, 10):  #adjusting labor hours by -20, -10, 0, 10, 20
    new_labor_hours_site2 = original_labor_hours_site2 + adjustment
    prob.constraints["Site2_Labor_Constraint"].changeRHS(new_labor_hours_site2)
    
    #solve
    prob.solve(GLPK_CMD(path=glpk_path))
    
    #output
    new_optimal_values_site2 = [product_vars_site1[i].varValue for i in range(len(products))]
    new_objective_value = pulp.value(prob.objective)
    
    print(f"\nAdjustment of {adjustment} labor hours at Site 2:")
    print("New Optimal Production Plan for Site 2:", new_optimal_values_site1)
    print("New Total Profit:", new_objective_value)


Adjustment of -20 labor hours at Site 2:
New Optimal Production Plan for Site 2: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 116921.48871499998

Adjustment of -10 labor hours at Site 2:
New Optimal Production Plan for Site 2: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 118947.98871499998

Adjustment of 0 labor hours at Site 2:
New Optimal Production Plan for Site 2: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 120974.44818499999

Adjustment of 10 labor hours at Site 2:
New Optimal Production Plan for Site 2: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 122499.026135

Adjustment of 20 labor hours at Site 2:
New Optimal Production Plan for Site 2: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 123639.028985


In [7]:
#Sensitivity Analysis re RSXX,STYY,CBAA and CARBON30 for Product 6

# Define raw materials, products, costs, usage, and inventories
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

def solve_problem(rsxx_inventory, styy_inventory, cbaa_inventory, carbon30_inventory):
    # Define the problem
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

    # Define decision variables for the quantity of each product to produce at each site
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]
    
    # Calculate profit values
    profit_values_site1 = []
    profit_values_site2 = []
    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)
    
    # Define profit calculations
    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]
    
    # Objective function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

    # Total labor hours
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

    sum_labor_site1 = pulp.lpSum(labor_hours_site1)
    sum_labor_site2 = pulp.lpSum(labor_hours_site2)

    prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
    prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

    # Inventory constraints
    for i in range(len(raw_materials)):
        # Site 1
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
        # Site 2
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

    # Adjustments for RSXX, STYY, CBAA, and CARBON 30
    prob.constraints["Site2_RSXX_Constraint"].changeRHS(rsxx_inventory)
    prob.constraints["Site2_STYY_Constraint"].changeRHS(styy_inventory)
    prob.constraints["Site2_CBAA_Constraint"].changeRHS(cbaa_inventory)
    prob.constraints["Site2_CARBON_30_Constraint"].changeRHS(carbon30_inventory)
    
    # Solve the problem
    glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  # Adjust path as needed
    status = prob.solve(GLPK_CMD(path=glpk_path))
    
    objective_value = pulp.value(prob.objective)
    optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    
    return prob, optimal_values_site2, objective_value

def sensitivity_analysis_product6_rsxx_styy_cbaa_carbon30():
    #product 6 index
    product_6_index = products.index('Product 6')

    #ranges for adjustements
    rsxx_range = range(0, 5001, 500)  
    styy_range = range(0, 1001, 250) 
    cbaa_range = range(0, 1001, 250) 
    carbon30_range = range(0, 1001, 250)  
    
    #sensitivity analysis
    for adj_RSXX in rsxx_range:
        for adj_STYY in styy_range:
            for adj_CBAA in cbaa_range:
                for adj_CARBON30 in carbon30_range:
                    #solve
                    prob, optimal_values_site2, objective_value = solve_problem(adj_RSXX, adj_STYY, adj_CBAA, adj_CARBON30)
                    
                    #output
                    print(f"Adjustments -> RSXX: {adj_RSXX}, STYY: {adj_STYY}, CBAA: {adj_CBAA}, CARBON 30: {adj_CARBON30}")
                    print(f"Optimal Production of Product 6: {optimal_values_site2[products.index('Product 6')]} units at Site 2")
                    print(f"New Total Profit: ${objective_value:,.2f}")
                    print("-" * 50)

#run
sensitivity_analysis_product6_rsxx_styy_cbaa_carbon30()


Adjustments -> RSXX: 0, STYY: 0, CBAA: 0, CARBON 30: 0
Optimal Production of Product 6: 0.0 units at Site 2
New Total Profit: $67,287.13
--------------------------------------------------
Adjustments -> RSXX: 0, STYY: 0, CBAA: 0, CARBON 30: 250
Optimal Production of Product 6: 0.0 units at Site 2
New Total Profit: $67,287.13
--------------------------------------------------
Adjustments -> RSXX: 0, STYY: 0, CBAA: 0, CARBON 30: 500
Optimal Production of Product 6: 0.0 units at Site 2
New Total Profit: $67,287.13
--------------------------------------------------
Adjustments -> RSXX: 0, STYY: 0, CBAA: 0, CARBON 30: 750
Optimal Production of Product 6: 0.0 units at Site 2
New Total Profit: $67,287.13
--------------------------------------------------
Adjustments -> RSXX: 0, STYY: 0, CBAA: 0, CARBON 30: 1000
Optimal Production of Product 6: 0.0 units at Site 2
New Total Profit: $67,287.13
--------------------------------------------------
Adjustments -> RSXX: 0, STYY: 0, CBAA: 250, CARBON 

In [8]:
#Sensitivity Analysis Optimizing RSXX and CARBON20 use in Product 9
# Define raw materials, products, costs, usage, and inventories
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

def solve_problem(rsxx_inventory, carbon20_inventory):
    # Define the problem
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

    # Define decision variables for the quantity of each product to produce at each site
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]
    
    # Calculate profit values
    profit_values_site1 = []
    profit_values_site2 = []
    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)
    
    # Define profit calculations
    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]
    
    # Objective function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

    # Total labor hours
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

    sum_labor_site1 = pulp.lpSum(labor_hours_site1)
    sum_labor_site2 = pulp.lpSum(labor_hours_site2)

    prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
    prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

    # Inventory constraints
    for i in range(len(raw_materials)):
        # Site 1
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
        # Site 2
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

    # Adjustments for RSXX and CARBON 20
    prob.constraints["Site2_RSXX_Constraint"].changeRHS(rsxx_inventory)
    prob.constraints["Site2_CARBON_20_Constraint"].changeRHS(carbon20_inventory)
    
    # Solve the problem
    glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  # Adjust path as needed
    status = prob.solve(GLPK_CMD(path=glpk_path))
    
    objective_value = pulp.value(prob.objective)
    optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    
    return prob, optimal_values_site2, objective_value

def sensitivity_analysis_product9_rsxx_carbon20():
    # Index for Product 9
    product_9_index = products.index('Product 9')

    # Define the range of adjustments for sensitivity analysis
    rsxx_range = range(0, 5001, 500)  # RSXX from 0 to 5000 in 500 unit increments
    carbon20_range = range(0, 1001, 250)  # CARBON 20 from 0 to 1000 in 250 unit increments

    # Perform sensitivity analysis by adjusting RSXX and CARBON 20 inventories
    for adj_RSXX in rsxx_range:
        for adj_CARBON20 in carbon20_range:
            # Solve the problem with the new constraints
            prob, optimal_values_site2, objective_value = solve_problem(adj_RSXX, adj_CARBON20)
            
            # Output results
            print(f"Adjustments -> RSXX: {adj_RSXX}, CARBON 20: {adj_CARBON20}")
            print(f"Optimal Production of Product 9: {optimal_values_site2[products.index('Product 9')]} units at Site 2")
            print(f"New Total Profit: ${objective_value:,.2f}")
            print("-" * 50)

# Run the sensitivity analysis
sensitivity_analysis_product9_rsxx_carbon20()


Adjustments -> RSXX: 0, CARBON 20: 0
Optimal Production of Product 9: 0.0 units at Site 2
New Total Profit: $77,688.02
--------------------------------------------------
Adjustments -> RSXX: 0, CARBON 20: 250
Optimal Production of Product 9: 0.0 units at Site 2
New Total Profit: $77,688.02
--------------------------------------------------
Adjustments -> RSXX: 0, CARBON 20: 500
Optimal Production of Product 9: 0.0 units at Site 2
New Total Profit: $77,688.02
--------------------------------------------------
Adjustments -> RSXX: 0, CARBON 20: 750
Optimal Production of Product 9: 0.0 units at Site 2
New Total Profit: $77,688.02
--------------------------------------------------
Adjustments -> RSXX: 0, CARBON 20: 1000
Optimal Production of Product 9: 0.0 units at Site 2
New Total Profit: $77,688.02
--------------------------------------------------
Adjustments -> RSXX: 500, CARBON 20: 0
Optimal Production of Product 9: 0.0 units at Site 2
New Total Profit: $95,991.31
--------------------

In [9]:
#Sensitivity Analysis for CARBON10 Availability at Site 1
#Sesitivity report showed that the increase from 10 to 1500 would fully utilize the constraint.
#Took original problem and looped thorugh inventory changes. 

#materails
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

#data
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

#raw materials
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

#labor
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

def solve_problem(carbon_10_inventory):
    #problem
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)
    
    #inventory changes
    updated_site1_inventory = site1_inventory.copy()
    updated_site2_inventory = site2_inventory.copy()
    updated_site1_inventory[6] = carbon_10_inventory
    updated_site2_inventory[6] = carbon_10_inventory
    
    #decision variables
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]
    
    #profit
    profit_values_site1 = []
    profit_values_site2 = []
    
    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)
    
    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]
    
    #obj function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"
    
    #labor constraint
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]
    
    sum_labor_site1 = pulp.lpSum(labor_hours_site1)
    sum_labor_site2 = pulp.lpSum(labor_hours_site2)
    
    prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
    prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"
    
    #inventory availability constraint per site
    for i in range(len(raw_materials)):
        #site 1
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
        #site 2
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"
    
    #solve
    glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  
    status = prob.solve(GLPK_CMD(path=glpk_path))
    
    objective_value = pulp.value(prob.objective)
    optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
    optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    
    return objective_value, optimal_values_site1, optimal_values_site2

#Sensitivity Analysis for CARBON10
carbon_10_availabilities = [300, 600, 1000, 1250, 1500, 2000]
results = {}

for availability in carbon_10_availabilities:
    objective_value, optimal_values_site1, optimal_values_site2 = solve_problem(availability)
    results[availability] = {
        'Objective Value': objective_value,
        'Optimal Values Site 1': optimal_values_site1,
        'Optimal Values Site 2': optimal_values_site2
    }

#output
for availability, result in results.items():
    print(f"CARBON 10 Availability: {availability}")
    print(f" New Total Profit: {result['Objective Value']}")
    print(f"Optimal Values Site 1: {result['Optimal Values Site 1']}")
    print(f"Optimal Values Site 2: {result['Optimal Values Site 2']}")
    print()



CARBON 10 Availability: 300
 New Total Profit: 128087.854998
Optimal Values Site 1: [0.0, 0.0, 54.0541, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 10 Availability: 600
 New Total Profit: 130584.06409999999
Optimal Values Site 1: [0.0, 0.0, 108.108, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 10 Availability: 1000
 New Total Profit: 133912.34905999998
Optimal Values Site 1: [0.0, 0.0, 180.18, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 10 Availability: 1250
 New Total Profit: 135992.52716
Optimal Values Site 1: [0.0, 0.0, 225.225, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 10 Availability: 1500
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 

In [10]:
#Sensitivity Analysis for CARBON20 
#Sensitivity report showed that the adjusting to fully utilize CARBON20 could increase profit and effective resource use

import pulp
from pulp import GLPK_CMD

# Materials
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Data
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Raw materials usage
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

#Labor
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

def solve_problem(carbon_20_inventory):
    #prob
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)
    
    #inventory changes
    updated_site1_inventory = site1_inventory.copy()
    updated_site2_inventory = site2_inventory.copy()
    updated_site1_inventory[8] = carbon_20_inventory
    updated_site2_inventory[8] = carbon_20_inventory
    
    # Decision variables
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]
    
    #profit
    profit_values_site1 = []
    profit_values_site2 = []
    
    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)
    
    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]
    
    #obj function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"
    
    #labor constraint
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]
    
    sum_labor_site1 = pulp.lpSum(labor_hours_site1)
    sum_labor_site2 = pulp.lpSum(labor_hours_site2)
    
    prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
    prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"
    
    #material availability
    for i in range(len(raw_materials)):
        # Site 1
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
        # Site 2
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"
    
    # Solve
    glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  
    status = prob.solve(GLPK_CMD(path=glpk_path))
    
    objective_value = pulp.value(prob.objective)
    optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
    optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    
    return objective_value, optimal_values_site1, optimal_values_site2

#sensitvity analysis
carbon_20_availabilities = [1000, 1500, 2000, 2500, 3000, 3500]
results = {}

for availability in carbon_20_availabilities:
    objective_value, optimal_values_site1, optimal_values_site2 = solve_problem(availability)
    results[availability] = {
        'Objective Value': objective_value,
        'Optimal Values Site 1': optimal_values_site1,
        'Optimal Values Site 2': optimal_values_site2
    }

#output
for availability, result in results.items():
    print(f"CARBON 20 Availability: {availability}")
    print(f" New Total Profit: {result['Objective Value']}")
    print(f"Optimal Values Site 1: {result['Optimal Values Site 1']}")
    print(f"Optimal Values Site 2: {result['Optimal Values Site 2']}")
    print()


CARBON 20 Availability: 1000
 New Total Profit: 138511.792105
Optimal Values Site 1: [0.0, 0.0, 212.014, 0.0, 0.0, 0.0, 0.0, 190.476, 230.88]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 20 Availability: 1500
 New Total Profit: 144145.70071499998
Optimal Values Site 1: [0.0, 0.0, 164.601, 0.0, 0.0, 0.0, 0.0, 285.714, 194.805]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 20 Availability: 2000
 New Total Profit: 147751.018903
Optimal Values Site 1: [0.0, 0.0, 73.2601, 0.0, 0.0, 0.0, 0.0, 380.952, 158.73]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 20 Availability: 2500
 New Total Profit: 149583.494785
Optimal Values Site 1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 444.444, 134.68]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 20 Availability: 3000
 New Total Profit: 149583.494785
Optimal Values Site 1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [11]:
#Sensitivity Analysis for CARBON30 


import pulp
from pulp import GLPK_CMD

# Materials
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Data
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Raw materials usage
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

#Labor
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

def solve_problem(carbon_30_inventory):
    #prob
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)
    
    #inventory changes
    updated_site1_inventory = site1_inventory.copy()
    updated_site2_inventory = site2_inventory.copy()
    updated_site1_inventory[9] = carbon_30_inventory
    updated_site2_inventory[9] = carbon_30_inventory
    
    # Decision variables
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]
    
    #profit
    profit_values_site1 = []
    profit_values_site2 = []
    
    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)
    
    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]
    
    #obj function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"
    
    #labor constraint
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]
    
    sum_labor_site1 = pulp.lpSum(labor_hours_site1)
    sum_labor_site2 = pulp.lpSum(labor_hours_site2)
    
    prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
    prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"
    
    #material availability
    for i in range(len(raw_materials)):
        # Site 1
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
        # Site 2
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"
    
    # Solve
    glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  
    status = prob.solve(GLPK_CMD(path=glpk_path))
    
    objective_value = pulp.value(prob.objective)
    optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
    optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    
    return objective_value, optimal_values_site1, optimal_values_site2

#sensitvity analysis
carbon_30_availabilities = [500, 1000, 1500, 2000, 2500, 3000]
results = {}

for availability in carbon_30_availabilities:
    objective_value, optimal_values_site1, optimal_values_site2 = solve_problem(availability)
    results[availability] = {
        'Objective Value': objective_value,
        'Optimal Values Site 1': optimal_values_site1,
        'Optimal Values Site 2': optimal_values_site2
    }

#output
for availability, result in results.items():
    print(f"CARBON 30 Availability: {availability}")
    print(f" New Total Profit: {result['Objective Value']}")
    print(f"Optimal Values Site 1: {result['Optimal Values Site 1']}")
    print(f"Optimal Values Site 2: {result['Optimal Values Site 2']}")
    print()

CARBON 30 Availability: 500
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 30 Availability: 1000
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 30 Availability: 1500
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 30 Availability: 2000
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

CARBON 30 Availability: 2500
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 1

In [12]:
#sensitivity Analysis for STYY for Site 1
import pulp
from pulp import GLPK_CMD

# Materials
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Data
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Raw Materials Usage
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

# Labor
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

def solve_problem(styy_inventory):
    # Problem
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)
    
    # Inventory changes
    updated_site1_inventory = site1_inventory.copy()
    updated_site1_inventory[1] = styy_inventory  # Update STYY inventory for site 1
    
    # Decision variables
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]
    
    # Profit calculation
    profit_values_site1 = []
    profit_values_site2 = []
    
    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)
    
    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]
    
    # Objective function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"
    
    # Labor constraint
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]
    
    prob += pulp.lpSum(labor_hours_site1) <= 168, "Site1_Labor_Constraint"
    prob += pulp.lpSum(labor_hours_site2) <= 168, "Site2_Labor_Constraint"
    
    # Inventory availability constraint per site
    for i in range(len(raw_materials)):
        # Site 1
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= updated_site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
        # Site 2
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"
    
    # Solve
    glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  
    status = prob.solve(GLPK_CMD(path=glpk_path))
    
    objective_value = pulp.value(prob.objective)
    optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
    optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    
    return objective_value, optimal_values_site1, optimal_values_site2

# Sensitivity Analysis for STYY
styy_availabilities = [1000, 2000, 3000, 4000, 5000]
results = {}

for availability in styy_availabilities:
    objective_value, optimal_values_site1, optimal_values_site2 = solve_problem(availability)
    results[availability] = {
        'Objective Value': objective_value,
        'Optimal Values Site 1': optimal_values_site1,
        'Optimal Values Site 2': optimal_values_site2
    }

# Output
for availability, result in results.items():
    print(f"Adjustment of STYY Availability: {availability}")
    print(f" New Total Profit: {result['Objective Value']}")
    print(f"Optimal Values Site 1: {result['Optimal Values Site 1']}")
    print(f"Optimal Values Site 2: {result['Optimal Values Site 2']}")
    print()


Adjustment of STYY Availability: 1000
 New Total Profit: 128230.49578199998
Optimal Values Site 1: [0.0, 0.0, 57.1429, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

Adjustment of STYY Availability: 2000
 New Total Profit: 134387.81834
Optimal Values Site 1: [0.0, 0.0, 190.476, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

Adjustment of STYY Availability: 3000
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

Adjustment of STYY Availability: 4000
 New Total Profit: 136258.24688
Optimal Values Site 1: [0.0, 0.0, 230.979, 0.0, 0.0, 0.0, 0.0, 152.381, 245.31]
Optimal Values Site 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107.451, 372.549]

Adjustment of STYY Availability: 5000
 New Total Profit: 136258.24688
Optimal Valu

In [13]:
#Sensitivity Analysis on Raw Material Inventory/Availability (RSXX)

#RSXX at Site 1
original_inventory_RSXX = site1_inventory[0]

#adjust inventory
for adjustment in range(-500, 4000, 500): 
    new_inventory_RSXX = original_inventory_RSXX + adjustment
    prob.constraints["Site1_RSXX_Constraint"].changeRHS(new_inventory_RSXX)
    
    #solve
    prob.solve(GLPK_CMD(path=glpk_path))
    
    #output
    new_optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
    new_objective_value = pulp.value(prob.objective)
    
    print(f"\nAdjustment of {adjustment} units of RSXX inventory at Site 1:")
    print("New Optimal Production Plan for Site 1:", new_optimal_values_site1)
    print("New Total Profit:", new_objective_value)



Adjustment of -500 units of RSXX inventory at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 226.852, 0.0, 0.0, 47.2491, 0.0, 152.381, 156.863]
New Total Profit: 119935.28663249999

Adjustment of 0 units of RSXX inventory at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 169.059, 0.0, 0.0, 110.142, 0.0, 152.381, 156.863]
New Total Profit: 123639.028985

Adjustment of 500 units of RSXX inventory at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 111.265, 0.0, 0.0, 173.035, 0.0, 152.381, 156.863]
New Total Profit: 127342.73529

Adjustment of 1000 units of RSXX inventory at Site 1:
New Optimal Production Plan for Site 1: [0.0, 0.0, 53.4713, 0.0, 0.0, 235.928, 0.0, 152.381, 156.863]
New Total Profit: 131046.455449

Adjustment of 1500 units of RSXX inventory at Site 1:
New Optimal Production Plan for Site 1: [0.799384, 0.0, 33.8939, 66.9037, 0.0, 256.41, 0.0, 152.381, 156.863]
New Total Profit: 133211.3458761

Adjustment of 2000 units of RSXX inventory at

In [14]:
#Sensitivity Analysis on Raw Material Inventory/Availability (RSXX) at Site 2

#RSXX at Site 2
original_inventory_RSXX = site2_inventory[0]

#adjust inventory
for adjustment in range(-500, 5500, 500): 
    new_inventory_RSXX = original_inventory_RSXX + adjustment
    prob.constraints["Site1_RSXX_Constraint"].changeRHS(new_inventory_RSXX)
    
    #solve
    prob.solve(GLPK_CMD(path=glpk_path))
    
    #output
    new_optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
    new_objective_value = pulp.value(prob.objective)
    
    print(f"\nAdjustment of {adjustment} units of RSXX inventory at Site 2:")
    print("New Optimal Production Plan for Site 2:", new_optimal_values_site2)
    print("New Total Profit:", new_objective_value)



Adjustment of -500 units of RSXX inventory at Site 2:
New Optimal Production Plan for Site 2: [44.749, 0.0, 0.0, 0.0, 0.0, 128.205, 0.0, 152.381, 156.863]
New Total Profit: 133211.3458761

Adjustment of 0 units of RSXX inventory at Site 2:
New Optimal Production Plan for Site 2: [44.749, 0.0, 0.0, 0.0, 0.0, 128.205, 0.0, 152.381, 156.863]
New Total Profit: 133242.8161525

Adjustment of 500 units of RSXX inventory at Site 2:
New Optimal Production Plan for Site 2: [44.749, 0.0, 0.0, 0.0, 0.0, 128.205, 0.0, 152.381, 156.863]
New Total Profit: 133242.8161525

Adjustment of 1000 units of RSXX inventory at Site 2:
New Optimal Production Plan for Site 2: [44.749, 0.0, 0.0, 0.0, 0.0, 128.205, 0.0, 152.381, 156.863]
New Total Profit: 133242.8161525

Adjustment of 1500 units of RSXX inventory at Site 2:
New Optimal Production Plan for Site 2: [44.749, 0.0, 0.0, 0.0, 0.0, 128.205, 0.0, 152.381, 156.863]
New Total Profit: 133242.8161525

Adjustment of 2000 units of RSXX inventory at Site 2:
New 

In [15]:
#Sensitivity Analysis - Changing Product Mix 1 to increase RSXX by 5%
#does not change anything
import pulp
from pulp import LpProblem, LpMaximize, LpVariable, GLPK_CMD, LpStatus

# Define raw materials and products
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Define data as arrays
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Define raw material usage per product (rows: raw materials, columns: products)
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

# Define labor hours per product at each site
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

# Define the problem
def create_problem():
    prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

    # Define decision variables for the quantity of each product to produce at each site
    product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
    product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]

    # Calculate profit per product per site
    profit_values_site1 = []
    profit_values_site2 = []

    for i in range(len(products)):
        cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
        profit_values_site1.append(product_prices[i] - cost)
        profit_values_site2.append(product_prices[i] - cost)

    profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
    profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]

    # Objective function
    prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

    # Calculate total labor hours per product per site
    labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
    labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

    # Sum the total labor hours for each site
    sum_labor_site1 = pulp.lpSum(labor_hours_site1)
    sum_labor_site2 = pulp.lpSum(labor_hours_site2)

    # Add labor hours constraints
    prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
    prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

    # Add inventory constraints
    for i in range(len(raw_materials)):
        prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

    return prob, product_vars_site1, product_vars_site2


prob, product_vars_site1, product_vars_site2 = create_problem()

#GLPK path
glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe"  

# Solve the original problem
prob.solve(GLPK_CMD(path=glpk_path))
original_objective_value = pulp.value(prob.objective)

print(f"Original Profit: {original_objective_value}")

#sensitvity analysis
rsxx_index = raw_materials.index('RSXX')
product_1_index = products.index('Product 1')

#new availability
adjustment_percentage = 0.05
original_usage = raw_material_usage[rsxx_index][product_1_index]
adjusted_usage = original_usage * (1 + adjustment_percentage)

print(f"Original RSXX usage for Product 1: {original_usage}")
print(f"Adjusted RSXX usage for Product 1: {adjusted_usage}")

raw_material_usage[rsxx_index][product_1_index] = adjusted_usage

#new problem
new_prob, new_product_vars_site1, new_product_vars_site2 = create_problem()

#solve
new_prob.solve(GLPK_CMD(path=glpk_path))
new_objective_value = pulp.value(new_prob.objective)

#output
print(f"New Total Profit", new_objective_value)


Original Profit: 136258.24688
Original RSXX usage for Product 1: 0.5
Adjusted RSXX usage for Product 1: 0.525
New Total Profit 136258.24688


In [16]:
#Labor Cost Senstivity Analysis Related to Hypothesis 1
#tracks with model and 168 hour value profit with labor costs
import pulp

# Define raw materials and products
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Define data as arrays
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 50, 100, 50, 80, 150, 50, 175, 175]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Define raw material usage per product (rows: raw materials, columns: products)
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

# Define labor hours per product at each site
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

# Define base labor constraint and labor costs to test
base_labor_constraint = 168  # Assuming base constraint is 168 hours
labor_costs = range(100, 401, 50)  # Labor costs from 100 to 400 in 50-hour steps

def labor_cost_sensitivity_analysis(labor_costs, site_choice):
    results = []

    for labor_cost in labor_costs:
        # Define the problem
        prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

        # Define decision variables for the quantity of each product to produce at each site
        product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
        product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]

        # Calculate profit per product per site
        profit_values_site1 = []
        profit_values_site2 = []

        for i in range(len(products)):
            cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
            profit_values_site1.append(product_prices[i] - cost)
            profit_values_site2.append(product_prices[i] - cost)

        profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
        profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]

        # Objective function
        prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

        # Calculate total labor hours per product per site
        labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
        labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

        # Sum the total labor hours for each site
        sum_labor_site1 = pulp.lpSum(labor_hours_site1)
        sum_labor_site2 = pulp.lpSum(labor_hours_site2)

        # Add labor hours constraints based on site choice and labor cost
        if site_choice == 1:
            prob += sum_labor_site1 <= labor_cost, "Site1_Labor_Constraint"
            prob += sum_labor_site2 <= base_labor_constraint, "Site2_Labor_Constraint"
        else:
            prob += sum_labor_site1 <= base_labor_constraint, "Site1_Labor_Constraint"
            prob += sum_labor_site2 <= labor_cost, "Site2_Labor_Constraint"

        # Calculate inventory constraints per raw material per site
        for i in range(len(raw_materials)):
            # Constraint for Site 1
            prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
            # Constraint for Site 2
            prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

        # Solve the problem using GLPK
        glpk_path = "C:\glpk-4.65\w64\glpsol.exe"  # Ensure you plug in your own path
        status = prob.solve(GLPK_CMD(path=glpk_path, options=['--ranges', 'Product_Optimization.sen']))

        # Output results
        objective_value = pulp.value(prob.objective)

        # Store results
        results.append((labor_cost, objective_value))

    return results

# Sensitivity analysis for Site 1
print("Labor Cost Sensitivity Analysis for Site 1:")
results_site1 = labor_cost_sensitivity_analysis(labor_costs, site_choice=1)
for labor_cost, profit in results_site1:
    print(f"Labor Cost: {labor_cost} hours - Total Profit: {profit}")

# Sensitivity analysis for Site 2
print("\nLabor Cost Sensitivity Analysis for Site 2:")
results_site2 = labor_cost_sensitivity_analysis(labor_costs, site_choice=2)
for labor_cost, profit in results_site2:
    print(f"Labor Cost: {labor_cost} hours - Total Profit: {profit}")


Labor Cost Sensitivity Analysis for Site 1:
Labor Cost: 100 hours - Total Profit: 127286.14249
Labor Cost: 150 hours - Total Profit: 133883.30184
Labor Cost: 200 hours - Total Profit: 138072.70526
Labor Cost: 250 hours - Total Profit: 138072.70526
Labor Cost: 300 hours - Total Profit: 138072.70526
Labor Cost: 350 hours - Total Profit: 138072.70526
Labor Cost: 400 hours - Total Profit: 138072.70526

Labor Cost Sensitivity Analysis for Site 2:
Labor Cost: 100 hours - Total Profit: 108869.94495499998
Labor Cost: 150 hours - Total Profit: 129198.38581499999
Labor Cost: 200 hours - Total Profit: 145724.040055
Labor Cost: 250 hours - Total Profit: 153682.44843
Labor Cost: 300 hours - Total Profit: 158300.44843
Labor Cost: 350 hours - Total Profit: 158310.93128999998
Labor Cost: 400 hours - Total Profit: 158310.93128999998


In [17]:
#Senstitivity Aanalysis for Under-utilized Materitals 

#define sensitvity analysis
def sensitivity_analysis(material_index, base_inventory, increments):
    results = []

    for increment in increments:
        #adjust material availability/inventory
        adjusted_inventory = base_inventory.copy()
        adjusted_inventory[material_index] = increment

        #prob
        prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

        # Define decision variables for the quantity of each product to produce at each site
        product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
        product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]

        #profit
        profit_values_site1 = []
        profit_values_site2 = []

        for i in range(len(products)):
            cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
            profit_values_site1.append(product_prices[i] - cost)
            profit_values_site2.append(product_prices[i] - cost)

        profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
        profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]

        #obj function
        prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

        #total labor hours
        labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
        labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

        sum_labor_site1 = pulp.lpSum(labor_hours_site1)
        sum_labor_site2 = pulp.lpSum(labor_hours_site2)

        prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
        prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

        #availability/inventory constraints
        for i in range(len(raw_materials)):
            #site 1
            prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= adjusted_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
        
            # site 2
            prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= adjusted_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

        #solve
        glpk_path = "C:\\glpk-4.65\\w64\\glpsol.exe" # Ensure you plug in your own path
        status = prob.solve(GLPK_CMD(path=glpk_path, options=['--ranges', 'Product_Optimization.sen']))

        #output
        objective_value = pulp.value(prob.objective)

        results.append((increment, objective_value))

    return results

#base
base_inventory = site1_inventory  #site 1
increments = range(0, 1001, 250)  #250 unit increments for utilization

#analysis for each material by increment change
materials = ['CBZZ', 'SBRXL', 'SLSM', 'CARBON 40', 'CARBON 60']
for material in materials:
    material_index = raw_materials.index(material)
    print(f"\nSensitivity Analysis for {material}:")
    results = sensitivity_analysis(material_index, base_inventory, increments)
    for increment, profit in results:
        print(f"Inventory Increment: {increment} units - Total Profit: {profit}")



Sensitivity Analysis for CBZZ:
Inventory Increment: 0 units - Total Profit: 126568.36468599999
Inventory Increment: 250 units - Total Profit: 126568.36468599999
Inventory Increment: 500 units - Total Profit: 126568.36468599999
Inventory Increment: 750 units - Total Profit: 126568.36468599999
Inventory Increment: 1000 units - Total Profit: 126568.36468599999

Sensitivity Analysis for SBRXL:
Inventory Increment: 0 units - Total Profit: 126568.36468599999
Inventory Increment: 250 units - Total Profit: 126568.36468599999
Inventory Increment: 500 units - Total Profit: 126568.36468599999
Inventory Increment: 750 units - Total Profit: 126568.36468599999
Inventory Increment: 1000 units - Total Profit: 126568.36468599999

Sensitivity Analysis for SLSM:
Inventory Increment: 0 units - Total Profit: 126568.36468599999
Inventory Increment: 250 units - Total Profit: 126568.36468599999
Inventory Increment: 500 units - Total Profit: 126568.36468599999
Inventory Increment: 750 units - Total Profit: 12